In [1]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import DPOTrainer
import bitsandbytes as bnb
import torch 
device = "cuda" if torch.cuda.is_available() else "cpu"
finetuned_model = "berkouille/assistant_DPO_4"


model = AutoModelForCausalLM.from_pretrained(
    finetuned_model,
    torch_dtype=torch.bfloat16,
    device_map='auto'
)

tokenizer = AutoTokenizer.from_pretrained(finetuned_model)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
def format_instruction(sample):
    return f"""You are a helpful putting assistant that helps golfers to improve their putting skills. You have two tasks. The first task is responding to users' questions with detailed answers to help them learn putting fundamentals and techniques. Your second task is opening the requested exercises for users on the green platform.
        ### Instruction:{sample} ### Response:"""

In [3]:

question = [
    "who are you?",
    "what are the putting fundamentals?",
    "Could you recommend me a good drill to practice it?",
    "what is ladder drills for?",
    "how can i improve my putting skills?",
    "what is the most important thing to do when putting?",
    "what kind of visual cues puttview provides for green reading?",
    "I am pretty bad at speed control, can you help me?",
    "What is the most popular golf course in the world?",
    "What are the best games in PuttView to practice distance accuracy?",
    "Is there any solution from PuttView for outdoors?",
    "I have a very small apartment, which puttview product should i get?",
    "What is the most expensive product of puttview?",
    "Who won the masters 2024?"
]

for query in question:
    prompt = format_instruction(query)
    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).to(device)
    
    with torch.inference_mode():
        outputs = model.generate(
            **input_ids, 
            max_new_tokens=200, 
            do_sample=True, 
            top_p=0.9,
            temperature=0.7
        )

    outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    #print("full output :", outputs)
    output = outputs[0][len(prompt):]
    print("------------------")
    print(output)